# Demo T1 processing notebook

## Configuration options


Please copy the link from OSF to your data:

In [ ]:
OSF_link = "https://osf.io/9j3pa/download/";

Please customize the inversion times (TI) for your measurements (in ms):

In [ ]:
TI = [50, 400, 1100, 2400];

Please specify your data type ("Magnitude" or "Complex"):

In [ ]:
dataType = "Magnitude"

In [ ]:
imageFile = '20191213_mhi_NIST/20191213_mhi_NIST_noFS.nii.gz'; % Path to image after unzipped

## Download data

In [ ]:
cmd = ['curl -L -o rrsg_dataset.zip', ' ', OSF_link];
[STATUS,MESSAGE] = unix(cmd);
unzip('rrsg_dataset.zip', 'data/');

## Startup qMRLab

In [1]:
%% MATLAB/OCTAVE CODE
% Adds qMRLab to the path of the environment

try
    cd ../work/qMRLab
    startup
    cd ../../demo
catch
    error("qMRLab could not be started correctly.")
end

loading struct
loading io
loading statistics
loading optim
loading image


## Run data format check

In [ ]:
% Reset dataCheckPassed variable
dataCheckPassed = [];

try 
    %% Check that the files have the correct names 
    if exist(strcat('data/', imageFile))
         % File exists.
    else
       error("Your OSF data must contain a file called ir_data.nii.gz, and it doesn't.")
    end

    % Load files
    try
        [data, hdr] = nii_load(strcat('data/', imageFile),0,'linear');
    catch
        error("ir_data.nii.gz could not be loaded correctly.")
    end

    % Check the dimensions of the files
    if size(data, 4) ~= length(TI)
        error("The fourth dimension of your `data` variable should be the same size as the number of TIs, but it isn't.")
    end
    
    % If you got here, all tests passed
    dataCheckPassed = true;
catch err
    error(err)
    dataCheckPassed = false;
end

if dataCheckPassed
    disp("All data format check passed, you can continue to the next step of the script.")
else
    disp("At least one data format check did not pass. Please review our submission guidelines, and reupload your data to OSF.")
end

## Save data and configuration

In [ ]:
save -mat7-binary 'dataAndConfig.mat'

## Prepare variables for fitting

In [ ]:
!sh ../run_ir_rrsg.sh $LD_LIBRARY_PATH foo /home/jovyan/demo/dataAndConfig.mat output.mat

## Fit data

In [2]:
load('output.mat')

## Plot data

In [8]:
%% MATLAB/OCTAVE CODE
% Code used to re-orient the images to make pretty figures, and to assign variables with the axis lengths.

T1_map = squeeze(FitResults.T1);
xAxis = [0:size(T1_map,2)-1];
yAxis = [0:size(T1_map,1)-1];

% Raw MRI data at different TI values
TI_1 = squeeze(data(:,:,:,1));
TI_2 = squeeze(data(:,:,:,2));
TI_3 = squeeze(data(:,:,:,3));
TI_4 = squeeze(data(:,:,:,4));


In [10]:
%get T1_map --from Octave
%get TI_1 --from Octave
%get TI_2 --from Octave
%get TI_3 --from Octave
%get TI_4 --from Octave

%get xAxis --from Octave
%get yAxis --from Octave

# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML

from plotly import tools
trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_1,
                   colorscale='Greys',
                   showscale = False,
                   visible=False,
                   name = 'Signal')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_2,
                   colorscale='Greys',
                   showscale = False,
                   visible=False,
                   name = 'Signal')
trace3 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_3,
                   colorscale='Greys',
                   showscale = False,
                   visible=True,
                   name = 'Signal')
trace4 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_4,
                   colorscale='Greys',
                   visible=False,
                   showscale = False,
                   name = 'Signal')
trace5 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=T1_map,
                   colorscale='Portland',
                   xaxis='x2',
                   yaxis='y2',
                   visible=True,
                   name = 'T1 values (ms)')

data=[trace1, trace2, trace3, trace4, trace5]


updatemenus = list([
    dict(active=2,
         x = 0.12,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16
            ),
         buttons=list([   
            dict(label = '50 ms',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, True]},
                         ]),
            dict(label = '400 ms',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, True]},
                         ]),
            dict(label = '1100 ms',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, True]},
                         ]),
            dict(label = '2400 ms',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, True]},
                         ]),
        ]),
    )
])

layout = dict(
    width=560,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.1,
            y=1.15,
            showarrow=False,
            text='MR Image',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.84,
            y=1.15,
            showarrow=False,
            text='T<sub>1</sub> map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.22,
            y=1.15,
            showarrow=False,
            text='T<sub>1</sub> (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.02,
            y=-0.15,
            showarrow=False,
            text='TI:',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis = dict(range = [0,255], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 0.5]),
    yaxis = dict(range = [0,255], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.1, 0.9]),
    xaxis2 = dict(range = [0,255], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.5, 1]),
    yaxis2 = dict(range = [0,255], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.1, 0.9], anchor='x2'),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)


fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-heatmap', config = config)


---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
script_7209524164466426353 in <module>
      import matplotlib.pyplot as plt
----> import plotly.plotly as py
      import plotly.graph_objs as go
      import numpy as np
      from plotly import __version__

ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead.
